In [1]:
# PyTorch  2.1.0
# Python  3.10(ubuntu22.04)
# CUDA  12.1
!pip install transformers==4.33.2
!pip install sentencepiece
!pip install cpm_kernels
!pip install gradio==3.32.0
!pip install mdtex2html
!pip install icetk

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [2]:
#autodl: 
import os
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'
os.environ['HF_ENDPOINT']='https://hf-mirror.com/'

from transformers import AutoTokenizer, AutoModel
# AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, cache_dir="/root/chatglm6b-local")
# AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, cache_dir="/root/chatglm6b-local")
tokenizer = AutoTokenizer.from_pretrained("/root/chatglm6b-local/models--THUDM--chatglm-6b/snapshots/bf0f5cfb575eebebf9b655c5861177acfee03f16", trust_remote_code=True)
model = AutoModel.from_pretrained("/root/chatglm6b-local/models--THUDM--chatglm-6b/snapshots/bf0f5cfb575eebebf9b655c5861177acfee03f16", trust_remote_code=True).half().cuda()
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
#response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
#print(response)
response,  history = model.chat(tokenizer, "用Python编写1-10的求和", history=history)
print(response)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

The dtype of attention mask (torch.int64) is not bool


你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。
可以使用Python中的range函数来生成1-10的整数序列，然后使用sum函数来计算序列的和。以下是一个简单的Python代码示例：

```python
range(1, 11)
print(sum(range(1, 11)))
```

运行上述代码会输出以下结果：

```
10
```

因此，使用range函数生成1-10的整数序列，然后使用sum函数来计算序列的和即可得到1-10的求和结果。


In [1]:
from transformers import AutoModel, AutoTokenizer
import gradio as gr
import mdtex2html

tokenizer = AutoTokenizer.from_pretrained("/root/chatglm6b-local/models--THUDM--chatglm-6b/snapshots/bf0f5cfb575eebebf9b655c5861177acfee03f16", trust_remote_code=True)
model = AutoModel.from_pretrained("/root/chatglm6b-local/models--THUDM--chatglm-6b/snapshots/bf0f5cfb575eebebf9b655c5861177acfee03f16", trust_remote_code=True).half().cuda()
model = model.eval()

"""Override Chatbot.postprocess"""


def postprocess(self, y):
    if y is None:
        return []
    for i, (message, response) in enumerate(y):
        y[i] = (
            None if message is None else mdtex2html.convert((message)),
            None if response is None else mdtex2html.convert(response),
        )
    return y


gr.Chatbot.postprocess = postprocess


def parse_text(text):
    """copy from https://github.com/GaiZhenbiao/ChuanhuChatGPT/"""
    lines = text.split("\n")
    lines = [line for line in lines if line != ""]
    count = 0
    for i, line in enumerate(lines):
        if "```" in line:
            count += 1
            items = line.split('`')
            if count % 2 == 1:
                lines[i] = f'<pre><code class="language-{items[-1]}">'
            else:
                lines[i] = f'<br></code></pre>'
        else:
            if i > 0:
                if count % 2 == 1:
                    line = line.replace("`", "\`")
                    line = line.replace("<", "&lt;")
                    line = line.replace(">", "&gt;")
                    line = line.replace(" ", "&nbsp;")
                    line = line.replace("*", "&ast;")
                    line = line.replace("_", "&lowbar;")
                    line = line.replace("-", "&#45;")
                    line = line.replace(".", "&#46;")
                    line = line.replace("!", "&#33;")
                    line = line.replace("(", "&#40;")
                    line = line.replace(")", "&#41;")
                    line = line.replace("$", "&#36;")
                lines[i] = "<br>"+line
    text = "".join(lines)
    return text


def predict(input, chatbot, max_length, top_p, temperature, history):
    chatbot.append((parse_text(input), ""))
    for response, history in model.stream_chat(tokenizer, input, history, max_length=max_length, top_p=top_p,
                                               temperature=temperature):
        chatbot[-1] = (parse_text(input), parse_text(response))       

        yield chatbot, history


def reset_user_input():
    return gr.update(value='')


def reset_state():
    return [], []


with gr.Blocks() as demo:
    gr.HTML("""<h1 align="center">ChatGLM</h1>""")

    chatbot = gr.Chatbot()
    with gr.Row():
        with gr.Column(scale=4):
            with gr.Column(scale=12):
                user_input = gr.Textbox(show_label=False, placeholder="Input...", lines=10).style(
                    container=False)
            with gr.Column(min_width=32, scale=1):
                submitBtn = gr.Button("Submit", variant="primary")
        with gr.Column(scale=1):
            emptyBtn = gr.Button("Clear History")
            max_length = gr.Slider(0, 4096, value=2048, step=1.0, label="Maximum length", interactive=True)
            top_p = gr.Slider(0, 1, value=0.7, step=0.01, label="Top P", interactive=True)
            temperature = gr.Slider(0, 1, value=0.95, step=0.01, label="Temperature", interactive=True)

    history = gr.State([])

    submitBtn.click(predict, [user_input, chatbot, max_length, top_p, temperature, history], [chatbot, history],
                    show_progress=True)
    submitBtn.click(reset_user_input, [], [user_input])

    emptyBtn.click(reset_state, outputs=[chatbot, history], show_progress=True)

demo.queue().launch(share=True, inbrowser=True)
# error...just use latest gradio

/root/miniconda3/lib/python3.10/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/root/miniconda3/lib/python3.10/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Running on local URL:  http://127.0.0.1:7860


Task exception was never retrieved
future: <Task finished name='shzfja4y49b_1' coro=<Queue.process_events() done, defined at /root/miniconda3/lib/python3.10/site-packages/gradio/queueing.py:343> exception=1 validation error for PredictBody
event_id
  Field required [type=missing, input_value={'data': [], 'event_data'...on_hash': 'shzfja4y49b'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing>
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.10/site-packages/gradio/queueing.py", line 347, in process_events
    client_awake = await self.gather_event_data(event)
  File "/root/miniconda3/lib/python3.10/site-packages/gradio/queueing.py", line 220, in gather_event_data
    data, client_awake = await self.get_message(event, timeout=receive_timeout)
  File "/root/miniconda3/lib/python3.10/site-packages/gradio/queueing.py", line 456, in get_message
    return PredictBody(**data), True
  File "/root/miniconda3/lib/python3.10/si


Could not create share link. Please check your internet connection or our status page: https://status.gradio.app
